In [2]:
import pandas as pd
import numpy as np
from tslearn.utils import to_time_series,to_time_series_dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tslearn.neighbors import KNeighborsTimeSeriesClassifier,KNeighborsTimeSeries
from tslearn.svm import TimeSeriesSVC
from tslearn.preprocessing import TimeSeriesScalerMinMax
from sklearn.model_selection import GridSearchCV,StratifiedKFold,KFold
from sklearn.pipeline import Pipeline
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale,maxabs_scale


In [3]:
#initialize
colname= ['nose_x', 'nose_y' , 'l_eye_x', 'l_eye_y', 'r_eye_x', 'r_eye_y', 'l_ear_x', 'l_ear_y', 'r_ear_x', 'r_ear_y', 'l_shoulder_x', 'l_shoulder_y', 'r_shoulder_x', 'r_shoulder_y', 'l_elbow_x', 'l_elbow_y', 'r_elbow_x', 'r_elbow_y', 'l_wrist_x', 'l_wrist_y', 'r_wrist_x', 'r_wrist_y', 'l_hip_x', 'l_hip_y', 'r_hip_x', 'r_hip_y', 'l_knee_x', 'l_knee_y', 'r_knee_x', 'r_knee_y', 'l_ankle_x', 'l_ankle_y', 'r_ankle_x', 'r_ankle_y']
workouts= pd.DataFrame(columns=colname)
workouts.insert(0,'filename','')


# Methods to find euclidean distance and euler angle

In [4]:
def scale(df):
    df_d = df[['l_nose_xy','l_eye_xy','r_eye_xy','l_ear_xy','r_ear_xy','l_shoulder_xy','r_shoulder_xy','l_elbow_xy','r_elbow_xy','l_wrist_xy','r_wrist_xy','l_hip_xy','r_hip_xy','l_knee_xy','r_knee_xy','l_ankle_xy','r_ankle_xy']]
    min_d = df_d.min().min()    # df['Low'].min()
    max_d = df_d.max().max()    # df['High'].max()  
    df_d = (df_d - min_d) / (max_d- min_d) 
    for x in df.columns:
        if(x.endswith('_a')):
            df[x] =  maxabs_scale(df[x])
    return  df

In [5]:
def calc_euc_dist(df1,df2,filename):
                
                df1['l_nose_xy'] = np.sqrt( df1['nose_x'] + df1['nose_y'] )
                df1['l_eye_xy'] = np.sqrt( df1['l_eye_x'] + df1['l_eye_y'] )
                df1['r_eye_xy'] = np.sqrt( df1['r_eye_x'] + df1['r_eye_y'] )
                df1['l_ear_xy'] = np.sqrt( df1['l_ear_x'] + df1['l_ear_y'] )
                df1['r_ear_xy'] = np.sqrt( df1['r_ear_x'] + df1['r_ear_y'] )
                df1['l_shoulder_xy'] =   np.sqrt( df1['l_shoulder_x'] + df1['l_shoulder_y'] )
                df1['r_shoulder_xy'] =   np.sqrt( df1['r_shoulder_x'] + df1['r_shoulder_y'] )
                df1['l_elbow_xy'] =   np.sqrt( df1['l_elbow_x'] + df1['l_elbow_y'] )
                df1['r_elbow_xy'] =   np.sqrt( df1['r_elbow_x'] + df1['r_elbow_y'] )
                df1['l_wrist_xy'] =   np.sqrt( df1['l_wrist_x'] + df1['l_wrist_y'] )
                df1['r_wrist_xy'] =   np.sqrt( df1['r_wrist_x'] + df1['r_wrist_y'] )
                df1['l_hip_xy'] =   np.sqrt( df1['l_hip_x'] + df1['l_hip_y'] )
                df1['r_hip_xy'] =   np.sqrt( df1['r_hip_x'] + df1['r_hip_y'] )
                df1['l_knee_xy'] =   np.sqrt( df1['l_knee_x'] + df1['l_knee_y'] )
                df1['r_knee_xy'] =   np.sqrt( df1['r_knee_x'] + df1['r_knee_y'] )
                df1['l_ankle_xy'] =   np.sqrt( df1['l_ankle_x'] + df1['l_ankle_y'] )
                df1['r_ankle_xy'] =   np.sqrt( df1['r_ankle_x'] + df1['r_ankle_y'] )
                df1['l_nose_a'] = np.tan( df2['nose_y']/df2['nose_x'] )
                df1['l_eye_a'] = np.tan( df2['l_eye_y']/df2['l_eye_x'] )
                df1['r_eye_a'] = np.tan( df2['r_eye_y']/df2['r_eye_x'] )
                df1['l_ear_a'] = np.tan( df2['l_ear_y']/df2['l_ear_x'] )
                df1['r_ear_a'] = np.tan( df2['r_ear_y']/df2['r_ear_x'] )
                df1['l_shoulder_a'] =  np.tan( df2['l_shoulder_y']/df2['l_shoulder_x'] )
                df1['r_shoulder_a'] = np.tan( df2['r_shoulder_y']/df2['r_shoulder_x'] )
                df1['l_elbow_a'] = np.tan( df2['l_elbow_y']/df2['l_elbow_x'] )
                df1['r_elbow_a'] = np.tan( df2['r_elbow_y']/df2['r_elbow_x'] )
                df1['l_wrist_a'] = np.tan( df2['l_wrist_y']/df2['l_wrist_x'] )
                df1['r_wrist_a'] = np.tan( df2['r_wrist_y']/df2['r_wrist_x'] )
                df1['l_hip_a'] = np.tan( df2['l_hip_y']/df2['l_hip_x'] )
                df1['r_hip_a'] = np.tan( df2['r_hip_y']/df2['r_hip_x'] )
                df1['l_knee_a'] = np.tan( df2['l_knee_y']/df2['l_knee_x'] )
                df1['r_knee_a'] = np.tan( df2['r_knee_y']/df2['r_knee_x'] )
                df1['l_ankle_a'] = np.tan( df2['l_ankle_y']/df2['l_ankle_x'] )
                df1['r_ankle_a'] = np.tan( df2['r_ankle_y']/df2['r_ankle_x'] )
                df1 =scale(df1)
                df1['filename'] = filename
                return df1[['l_nose_xy','l_eye_xy','r_eye_xy','l_ear_xy','r_ear_xy','l_shoulder_xy','r_shoulder_xy','l_elbow_xy','r_elbow_xy','l_wrist_xy','r_wrist_xy','l_hip_xy','r_hip_xy','l_knee_xy','r_knee_xy','l_ankle_xy','r_ankle_xy','l_shoulder_a','r_shoulder_a','l_elbow_a','r_elbow_a','l_wrist_a','r_wrist_a','l_hip_a','r_hip_a','l_knee_a','r_knee_a','l_ankle_a','r_ankle_a','l_nose_a','l_ear_a','r_ear_a','l_eye_a','r_eye_a','filename']]
                

In [6]:
def coarse_df():
    workouts= pd.DataFrame()
    y=[]
    for root, dirs, files in os.walk('../Data/Detectron/Train/'):
        for name in files:
                if(name.endswith('.csv') == False):
                    continue
                filepath = root + os.sep + name
                worktype = filepath.split(os.sep )[-2]
                filename = filepath.split(os.sep)[-1].replace('.csv','')
                if(os.path.getsize(filepath) > 0):
                    df = pd.read_csv(filepath,names=colname,header=None)
                    # to find euclidean distance
                    df1 = df[::5].diff().pow(2)
                    df2 = df[::5].diff()
                    if(len(df) < 250):
                        workouts = pd.concat([workouts,calc_euc_dist(df1,df2,filename)])
                        y.append(worktype)
    return workouts,y

In [7]:
workouts,y = coarse_df()
workouts.dropna(inplace=True)
X=workouts.set_index('filename').groupby('filename').apply(pd.DataFrame.to_numpy).to_numpy()
encoder = LabelEncoder()
y = encoder.fit_transform(y)
# convert to time series dataset    
formatted_time_series = to_time_series_dataset(X)
X_train, X_test, y_train, y_test = train_test_split(formatted_time_series, y, test_size=0.3, random_state=42)

# Initialize KNN TimeSeries Classifier from tslearn library

In [8]:
knn = KNeighborsTimeSeriesClassifier(n_neighbors=2,weights='distance',metric='dtw')
knn.fit(X_train, y_train)

KNeighborsTimeSeriesClassifier(n_neighbors=2, weights='distance')

In [9]:
knn.score(X_train, y_train),knn.score(X_test, y_test),knn.score(X,y)

(1.0, 0.5625, 0.8679245283018868)

In [10]:
pipeline = GridSearchCV(
    Pipeline([
            ('knn', KNeighborsTimeSeriesClassifier())
    ]),
    {'knn__n_neighbors': [2,5,10, 25], 'knn__weights': ['uniform', 'distance']},
)

In [11]:
pipeline.fit(X_train, y_train)
pipeline.best_params_,pipeline.best_score_


({'knn__n_neighbors': 2, 'knn__weights': 'distance'}, 0.5657142857142856)

# Initialize SVN Classifier from Timeseries Library

In [13]:
svc = TimeSeriesSVC(kernel="gak", gamma="auto") 
svc.fit(X_train, y_train)

TimeSeriesSVC()

In [14]:
svc.score(X_train, y_train)

0.8918918918918919

In [15]:
svc.score(X_test, y_test)

0.34375

In [16]:
pipeline = GridSearchCV(
    Pipeline([
            ('svc', TimeSeriesSVC())
    ]),
    {'svc__C': [1, 10, 100, 1000], 'svc__kernel': ['gak'], 'svc__gamma': ['auto']},
)   

In [18]:
pipeline.fit(X_train, y_train)
pipeline.score(X_train, y_train)

1.0

In [19]:
pipeline.score(X_test, y_test)

0.5

In [20]:
pipeline.best_params_,pipeline.best_score_


({'svc__C': 100, 'svc__gamma': 'auto', 'svc__kernel': 'gak'},
 0.5276190476190477)